## Problem Statement/Goal



## Imports

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
from selenium import webdriver
import requests
from time import sleep
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split

In [2]:
def save_posts(list_of_dicts, saver, forum_page):
    return pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'nums_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'auth_img',
                            'title',
                            'forum_page']).to_csv(saver + str(forum_page), index=False)

In [3]:
def click_next():
    last_page = False
    try:
        nexts = []
        for element in browser.find_elements_by_class_name('Button-content'):
            if element.text == 'NEXT':
                nexts.append(element)
        nexts[1].click()
        last_page = True
    except:
        last_page = False
    return last_page

In [6]:
ops = webdriver.chrome.options.Options()
ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_pull2_'                                # Path to save

for forum_page in range(1000,2001):   # BE CAREFUL WITH THIS                                     # For each page in chunk of posts
    try:                                                                # Failsafe within forum page
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)                                       # Go to forum page
        title_list = browser.find_elements_by_class_name("ForumTopic-title")# Define list of topic pages to click
        if forum_page % 100 == 0 & forum_page != 1000:                                       # Failsafe saver per 100 pages
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                #print('start dict')
                while more_pages == True:                               # While true that there's more pages...
                    title      = browser.find_element_by_class_name('Topic-title').text
                    dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
                    words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
                    nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
                    post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
                    auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
                    prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    #print('dict components defined')
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                                'text'      : words[post],              # Text of the post
                                'date'      : dates[post],              # Date of the (unedited) post
                                'nums_dict' : nums_dict[post],          # Author info & votes of the post
                                'post_num'  : post_num[post],           # Post umber in the topic
                                'prof_link' : prof_link[post],          # Link to author profile
                                'auth_img'  : auth_img[post],           # Profile image of author
                                'title'     : title,                    # Title of Topic
                                'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    more_pages = click_next()                           # Clicks "NEXT" otherwise return False
                    #print('next clicked')
                browser.get(current_list)                               # Returns to current forum page
                sleep(0.5)                                              # Tiny safety sleep
                #print('slept')
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
                
            except:
                print("Woops", forum_page, i)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
    except:
        print("Woops", forum_page)                                      # Informs that there's a problem
        save_posts(list_of_dicts, saver, forum_page)                    # Save current posts
        sleep(30)                                                       # Long Safety Sleep
        browser.refresh()                                               # Safety Refresh
        browser.get(current_list)                                       # Return to forum page
        
save_posts(list_of_dicts, saver, forum_page)                            # Finally save the posts
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])             # Return the dataframe for looking! :D

Woops 1003 8
Woops 1003


TimeoutException: Message: timeout
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [7]:
path_to_chromedriver = '../../../chromedriver'
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

list_of_dicts = []
saver = './Seed Projects/data/Overwatch_Sample5kto10k_'

for forum_page in range(5000,10000,10):
    try:
        browser.get('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 5 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):
            try:
                title_list[i].click()                                     # Click on Title i
                soup = BeautifulSoup(browser.page_source, 'lxml')         # Soup of all
                words = soup.find_all('div',{"class" :'TopicPost-bodyContent'}) # Words
                dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})     # Dates
                sleep(0.5)

                for post in range(len(words)):
                    post_dict = {                                         # Creation & Statement of dicts
                        'text' : words[post].text,
                        'date' : dates[post].text
                    }
                    list_of_dicts.append(post_dict)

                browser.back()                                            # Back
                title_list = browser.find_elements_by_class_name("ForumTopic-title")
            except:
                print("Woops", forum_page, i)
                save_posts(list_of_dicts, saver, forum_page)
    except:
        print("Woops", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.back()
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

Woops 6350 33
Woops 6570 13
Woops 6570 14
Woops 6570 15
Woops 6570 16
Woops 6570 17
Woops 6570 18
Woops 6570 19
Woops 6570 20
Woops 6570 21
Woops 6570 22
Woops 6570 23
Woops 6570 24
Woops 6570 25
Woops 6570 26
Woops 6570 27
Woops 6570 28
Woops 6570 29
Woops 6570 30
Woops 6570 31
Woops 6570 32
Woops 6570 33
Woops 6570 34
Woops 6570 35
Woops 6570 36
Woops 6570 37
Woops 6570 38
Woops 6570 39
Woops 6570 40
Woops 6570 41
Woops 6570 42
Woops 6570 43
Woops 6570 44
Woops 6570 45
Woops 6570 46
Woops 6570 47
Woops 6570 48
Woops 6570 49
Woops 7250 2
Woops 7250 3
Woops 7250 4
Woops 7250 5
Woops 7250 6
Woops 7250 7
Woops 7250 8
Woops 7250 9
Woops 7250 10
Woops 7250 11
Woops 7250 12
Woops 7250 13
Woops 7250 14
Woops 7250 15
Woops 7250 16
Woops 7250 17
Woops 7250 18
Woops 7250 19
Woops 7250 20
Woops 7250


KeyboardInterrupt: 